In [3]:
# import the libraries required for data access, operations and display
import numpy as np
import pandas as pd
# allows displaying the data inline (DataFram display() function)
from IPython.display import display
# allows inline display of graphics
%matplotlib inline

# read the data from the csv file
data = pd.read_csv("C:\\Users\\anshul\\sp3Master.csv")

In [5]:
display(data.head(n=5))

,CAL_DATE,ITEM_NO,LOC,SR_FLAG,CRFA_C,CRFA_R,CRFA_F,CRFA_A,EOQ,PKG_QTY,OUTPROC_FLAG,SCORE_862_INSTABILITY_AVG,AVG_ADJ_QTY,DESTINID,AVG_DAYS_LATE
0,19-09-2017,000 446 08 75,65,True,S,9,1,2,1,48,False,100.0,-1,65,0.35
1,19-09-2017,04-28651-000,65,True,S,9,4,2,1,50,False,100.0,-1,65,0.35
2,19-09-2017,05-30811-001,65,True,S,9,1,2,1,6,False,100.0,-1,65,0.35
3,19-09-2017,06-53144-000,65,True,S,9,1,2,1,400,False,100.0,-880,65,0.35
4,19-09-2017,06-93145-000,65,True,S,9,4,2,1,0,False,100.0,-1,65,0.35


In [6]:
# split the data into feature set (X) and response variable(s) (y)
y = data['SR_FLAG']
X = data.drop('SR_FLAG', axis=1)

In [8]:
y = y.apply(lambda f: np.where(f==False, 0, 1))

In [11]:
X_numeric = ['LOC','CRFA_R', 'CRFA_F','CRFA_A','EOQ','PKG_QTY','SCORE_862_INSTABILITY_AVG','DESTINID','AVG_DAYS_LATE']
X_log_transformed = pd.DataFrame(data=X)
X_log_transformed[X_numeric] = X[X_numeric].apply(lambda f: np.log(f + 1))

In [12]:
display(X_log_transformed.head(n=5))

,CAL_DATE,ITEM_NO,LOC,CRFA_C,CRFA_R,CRFA_F,CRFA_A,EOQ,PKG_QTY,OUTPROC_FLAG,SCORE_862_INSTABILITY_AVG,AVG_ADJ_QTY,DESTINID,AVG_DAYS_LATE
0,19-09-2017,000 446 08 75,4.189655,S,2.302585,0.693147,1.098612,0.693147,3.891820,False,4.615121,-1,4.189655,0.300105
1,19-09-2017,04-28651-000,4.189655,S,2.302585,1.609438,1.098612,0.693147,3.931826,False,4.615121,-1,4.189655,0.300105
2,19-09-2017,05-30811-001,4.189655,S,2.302585,0.693147,1.098612,0.693147,1.945910,False,4.615121,-1,4.189655,0.300105
3,19-09-2017,06-53144-000,4.189655,S,2.302585,0.693147,1.098612,0.693147,5.993961,False,4.615121,-880,4.189655,0.300105
4,19-09-2017,06-93145-000,4.189655,S,2.302585,1.609438,1.098612,0.693147,0.000000,False,4.615121,-1,4.189655,0.300105


In [30]:
# use the MinMaxScaler Scaler function to scale the numeric features
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_log_scaled = pd.DataFrame(data=X_log_transformed)
X_log_scaled[X_numeric] = scaler.fit_transform(X_log_transformed[X_numeric])

In [38]:
display(X_log_scaled.head(n=5))

,LOC,CRFA_R,CRFA_F,CRFA_A,EOQ,PKG_QTY,OUTPROC_FLAG,SCORE_862_INSTABILITY_AVG,AVG_ADJ_QTY,DESTINID,...,CRFA_C_3,CRFA_C_B,CRFA_C_D,CRFA_C_E,CRFA_C_F,CRFA_C_G,CRFA_C_H,CRFA_C_K,CRFA_C_S,CRFA_C_Z
0,1.0,0.460299,0.000000,0.323657,0.0,0.422544,0,0.349242,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
1,1.0,0.460299,0.609205,0.323657,0.0,0.426888,0,0.349242,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
2,1.0,0.460299,0.000000,0.323657,0.0,0.211272,0,0.349242,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
3,1.0,0.460299,0.000000,0.323657,0.0,0.650779,0,0.349242,-880,1.0,...,0,0,0,0,0,0,0,0,1,0
4,1.0,0.460299,0.609205,0.323657,0.0,0.000000,0,0.349242,-1,1.0,...,0,0,0,0,0,0,0,0,1,0


In [37]:
# X_log_scaled = X_log_scaled.drop('CRFA_C', axis=1)
X_log_scaled['OUTPROC_FLAG']=X_log_scaled['OUTPROC_FLAG'].apply(lambda x: np.where(x==False,0,1))
# X_log_scaled['CRFA_C']=X_log_scaled['CRFA_C'].apply(lambda x: np.where(x=='S',1,0))

In [15]:
# Split the data into Train/Test sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_log_scaled, y, test_size=0.2, random_state=0)

In [29]:
# Pre-calculating the metrices for a naive model. 
# That is when the outcome is always predicted as true, how will the raw model behave.
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, matthews_corrcoef, classification_report

# predicting everything as True(1)
y_naive = [1]*len(y_test)

print('Accuracy: ', accuracy_score(y_naive, y_test))
print('Recall: ', recall_score(y_naive, y_test))
print('Precision: ', precision_score(y_naive, y_test))
print('F1-Score: ', f1_score(y_naive, y_test))
# print('Matthews Correlation Coefficient: ', matthews_corrcoef(y_naive, y_test))
print('Classification Report: \n', classification_report(y_naive, y_test, target_names=['Short'], labels=[1]))
# print ("Naive Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(accuracy, fscore))

## -------------------------------------- ##
## This entire block is incorrect/useless ##
## -------------------------------------- ##

Accuracy:  0.29897056691315066
Recall:  0.29897056691315066
Precision:  1.0
F1-Score:  0.46031923205714925
Classification Report: 
              precision    recall  f1-score   support

      Short       1.00      0.30      0.46     20691

avg / total       1.00      0.30      0.46     20691



In [33]:
# one-hot encoding
print("Total features before one-hot encoding: ", len(X_log_scaled.columns))
X_log_scaled = pd.get_dummies(data=X_log_scaled)
print("Total features after one-hot encoding: ", len(X_log_scaled.columns))

Total features before one-hot encoding:  14
Total features after one-hot encoding:  11603


In [39]:
# Implementing Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, fbeta_score

clf=LogisticRegression()

# components for GridSearchCV
parameters = {'C': [10**-i for i in range(-5, 5)], 'class_weight': [None, 'balanced']}
scorer = make_scorer(fbeta_score, beta=0.5)
grid_obj = GridSearchCV(estimator=clf,param_grid=parameters,scoring=scorer)
grid_fit = grid_obj.fit(X_train, y_train)
# print(X_train)
# print(y_train)
# select the best estimator
best_clf = grid_fit.best_estimator_

# make predictions using the unoptimized and optimized models
y_pred = clf.fit(X_train, y_train).predict(X_test)
y_best_pred = best_clf.predict(X_test)

ValueError: could not convert string to float: 'S'

# Implement the learnigs on DTML data

<b>Applying Classification using KNN</b>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# read the data from the csv file
data = pd.read_csv("C:\\Users\\anshul\\sp3Master.csv")
data.head()

,CAL_DATE,ITEM_NO,LOC,SR_FLAG,CRFA_C,CRFA_R,CRFA_F,CRFA_A,EOQ,PKG_QTY,OUTPROC_FLAG,SCORE_862_INSTABILITY_AVG,AVG_ADJ_QTY,DESTINID,AVG_DAYS_LATE
0,19-09-2017,000 446 08 75,65,True,S,9,1,2,1,48,False,100.0,-1,65,0.35
1,19-09-2017,04-28651-000,65,True,S,9,4,2,1,50,False,100.0,-1,65,0.35
2,19-09-2017,05-30811-001,65,True,S,9,1,2,1,6,False,100.0,-1,65,0.35
3,19-09-2017,06-53144-000,65,True,S,9,1,2,1,400,False,100.0,-880,65,0.35
4,19-09-2017,06-93145-000,65,True,S,9,4,2,1,0,False,100.0,-1,65,0.35


In [16]:
# everything other than the SR_FLAG is feature vector
# the SR_FLAG is the response variable

# assigning 'SR_FLAG' to the response vector 'y'
y = data['SR_FLAG']

# Using the dataframe.drop() function from the pandas library to 
# remove one column from the data source and create the feature vector set.
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html
# the first parameter to the function drop is the name of the column 
# and the second parameter 'axis' indicates whether we are referring to an index or an actual column name:
# 0 or ‘index’, 1 or ‘columns’
X = data.drop('SR_FLAG', axis='columns')

In [17]:
# y.head()
X.head()

,CAL_DATE,ITEM_NO,LOC,CRFA_C,CRFA_R,CRFA_F,CRFA_A,EOQ,PKG_QTY,OUTPROC_FLAG,SCORE_862_INSTABILITY_AVG,AVG_ADJ_QTY,DESTINID,AVG_DAYS_LATE
0,19-09-2017,000 446 08 75,65,S,9,1,2,1,48,False,100.0,-1,65,0.35
1,19-09-2017,04-28651-000,65,S,9,4,2,1,50,False,100.0,-1,65,0.35
2,19-09-2017,05-30811-001,65,S,9,1,2,1,6,False,100.0,-1,65,0.35
3,19-09-2017,06-53144-000,65,S,9,1,2,1,400,False,100.0,-880,65,0.35
4,19-09-2017,06-93145-000,65,S,9,4,2,1,0,False,100.0,-1,65,0.35


In [18]:
y.head()

0    True
1    True
2    True
3    True
4    True
Name: SR_FLAG, dtype: bool

In [19]:
# convert the response vector y to a boolean vector (1 for 'True' and 0 for 'False') using the apply function
y = y.apply(lambda f: np.where(f==False, 0, 1))
y.head()

0    1
1    1
2    1
3    1
4    1
Name: SR_FLAG, dtype: int64

In [21]:
# split the data into training and testing sets
from sklearn.model_selection import train_test_split

In [22]:
# splitting the data in to groups of 75/25 as train/test ratio with a fixed random_state of 0.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [23]:
# check the types of the records
print('X_train Type: ', type(X_train))
print('X_test Type: ', type(X_test))
print('y_train Type: ', type(y_train))
print('y_test Type: ', type(y_test))

X_train Type:  <class 'pandas.core.frame.DataFrame'>
X_test Type:  <class 'pandas.core.frame.DataFrame'>
y_train Type:  <class 'pandas.core.series.Series'>
y_test Type:  <class 'pandas.core.series.Series'>


In [24]:
# check the shapes of the records
print('X_train Shape: ', X_train.shape)
print('X_test Shape: ', X_test.shape)
print('y_train Shape: ', y_train.shape)
print('y_test Shape: ', y_test.shape)

X_train Shape:  (77589, 14)
X_test Shape:  (25864, 14)
y_train Shape:  (77589,)
y_test Shape:  (25864,)
